# Perfilamiento sobre algoritmo Simplex con Blas

### 1. Medición de tiempo

Características de la instancia que utilizamos para el perfilamiento

In [5]:
%%bash
lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  2
Core(s) per socket:  4
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz
Stepping:            4
CPU MHz:             3099.747
BogoMIPS:            5000.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            33792K
NUMA node0 CPU(s):   0-7
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hy

In [6]:
%%bash
sudo lshw -C memory

  *-memory
       description: System memory
       physical id: 0
       size: 30GiB


In [7]:
import math
import time
import Simplex
import numpy as np
from pytest import approx
from scipy.optimize import linprog

### Módulo time

Ejemplo Maximización para evualuar tiempo de ejecución

In [8]:
c = [3, 5]
b = [4, 12, 18]
A = [[1,  0],
    [0,  2],
    [3, 2]]

start_time = time.time()
problema = Simplex.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
end_time = time.time()
secs = end_time-start_time
print("Simplex algorithm tomó",secs,"segundos" )

Solution for x vector, optimization value and status:
Simplex algorithm tomó 0.0008554458618164062 segundos


Con Scipy

In [9]:
c_scipu=[-3, -5]

start_time = time.time()
opt = linprog(c=c_scipu, A_ub=A, b_ub=b,
              method="simplex")
end_time = time.time()
secs = end_time-start_time
print("Scipy  tomó",secs,"segundos" )

Scipy  tomó 0.08662891387939453 segundos


Obs. En esta primera parte nos damos cuenta que el algoritmo implementado es un poco más rápido que el de Scipy.

### Comando Magic %time

In [10]:
%time problema.solve()

Solution for x vector, optimization value and status:
CPU times: user 548 µs, sys: 118 µs, total: 666 µs
Wall time: 456 µs


([0, 0], 0, 0)

### Cprofile

Para poder visualizar en que secciones de código se tarda más

In [11]:
import cProfile

In [12]:
cprof = cProfile.Profile()
cprof.enable()
problema = Simplex.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
cprof.disable()
cprof.print_stats(sort='cumtime')

Solution for x vector, optimization value and status:
         310 function calls (308 primitive calls) in 0.001 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.001    0.000 interactiveshell.py:3288(run_code)
        3    0.000    0.000    0.001    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.001    0.001 <ipython-input-12-de806f07ed56>:4(<module>)
        1    0.000    0.000    0.001    0.001 __init__.py:45(solve)
     10/8    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(solve)
        4    0.000    0.000    0.000    0.000 linalg.py:314(solve)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        2    0.000    0.000    0.000    0.000 iostream.py:384(write)
        3    0.000    0.000    0.000  

In [13]:
import pstats

cprof.dump_stats("Simplex_stats")

In [14]:
p_simplex_stats = pstats.Stats("Simplex_stats")
print(p_simplex_stats.sort_stats("cumulative").print_stats(10))

Sun May  9 15:46:20 2021    Simplex_stats

         310 function calls (308 primitive calls) in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 83 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.001    0.000 /usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3288(run_code)
        3    0.000    0.000    0.001    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.001    0.001 <ipython-input-12-de806f07ed56>:4(<module>)
        1    0.000    0.000    0.001    0.001 /usr/local/lib/python3.6/dist-packages/Simplex/__init__.py:45(solve)
     10/8    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(solve)
        4    0.000    0.000    0.000    0.000 /usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py:314

In [15]:
print(p_simplex_stats.sort_stats("cumulative").print_stats("solve|module"))

Sun May  9 15:46:20 2021    Simplex_stats

         310 function calls (308 primitive calls) in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 83 to 7 due to restriction <'solve|module'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 <ipython-input-12-de806f07ed56>:4(<module>)
        1    0.000    0.000    0.001    0.001 /usr/local/lib/python3.6/dist-packages/Simplex/__init__.py:45(solve)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(solve)
        4    0.000    0.000    0.000    0.000 /usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py:314(solve)
        1    0.000    0.000    0.000    0.000 <ipython-input-12-de806f07ed56>:3(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-12-de806f07ed56>:5(<module>)
        4    0.000    0.000    0.000    0.000 /usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py:310(_solve_dispatcher)

In [16]:
#numero de llamadas a funciones primitivas
print(p_simplex_stats.prim_calls)

308


In [17]:
p_simplex_stats.strip_dirs().sort_stats("cumulative").print_callers(10)

   Ordered by: cumulative time
   List reduced from 83 to 10 due to restriction <10>

Function                                                                 was called by...
                                                                             ncalls  tottime  cumtime
interactiveshell.py:3288(run_code)                                       <- 
{built-in method builtins.exec}                                          <-       3    0.000    0.001  interactiveshell.py:3288(run_code)
<ipython-input-12-de806f07ed56>:4(<module>)                              <-       1    0.000    0.001  {built-in method builtins.exec}
__init__.py:45(solve)                                                    <-       1    0.000    0.001  <ipython-input-12-de806f07ed56>:4(<module>)
{built-in method numpy.core._multiarray_umath.implement_array_function}  <-       1    0.000    0.000  <__array_function__ internals>:2(atleast_1d)
                                                                             

In [18]:
p_simplex_stats.strip_dirs().sort_stats("cumulative").print_callees("print|dot|ndim|solve")

   Ordered by: cumulative time
   List reduced from 83 to 5 due to restriction <'print|dot|ndim|solve'>

Function                                 called...
                                             ncalls  tottime  cumtime
__init__.py:45(solve)                    ->       1    0.000    0.000  <__array_function__ internals>:2(copy)
                                                  1    0.000    0.000  <__array_function__ internals>:2(hstack)
                                                  2    0.000    0.000  <__array_function__ internals>:2(size)
                                                  4    0.000    0.000  <__array_function__ internals>:2(solve)
                                                  2    0.000    0.000  twodim_base.py:152(eye)
                                                  7    0.000    0.000  {built-in method builtins.len}
                                                  3    0.000    0.000  {built-in method builtins.max}
                                

### Lineprofiler

Para saber línea por línea en que parte nuestro algoritmo se está tardano más en ejecutar

In [19]:
import line_profiler

problema = Simplex.Simplex(c,A,b,problem='Max')
line_prof = line_profiler.LineProfiler()
print(line_prof(problema.solve)())

Solution for x vector, optimization value and status:
([2.0, 6.0], -36.0, 0)


In [20]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 0.00066 s
File: /usr/local/lib/python3.6/dist-packages/Simplex/__init__.py
Function: solve at line 45

Line #      Hits         Time  Per Hit   % Time  Line Contents
    45                                               def solve(self):
    46                                                   
    47                                                   """
    48                                                   Solves the simplex algorithm. 
    49                                                   Returns
    50                                                   -------
    51                                                   :solution: Numpy array with solution
    52                                           
    53                                                   """
    54         1          2.0      2.0      0.3          problem = self.problem
    55         1          1.0      1.0      0.2          verbose = self.verbose
    56         1          1.0